In [1]:
# Erasmus+ ICCT project (2018-1-SI01-KA203-047081)

# Toggle cell visibility

from IPython.display import HTML
tag = HTML('''<script>
code_show=true; 
function code_toggle() {
    if (code_show){
        $('div.input').hide()
    } else {
        $('div.input').show()
    }
    code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
Promijeni vidljivost <a href="javascript:code_toggle()">ovdje</a>.''')
display(tag)

# Hide the code completely

# from IPython.display import HTML
# tag = HTML('''<style>
# div.input {
#     display:none;
# }
# </style>''')
# display(tag)


## Formulacija prostora stanja i prijenosna funkcija 

Ovaj primjer omogućava istraživanje učinaka strukturnih svojstava (upravljivost i osmotrivost) na prijenosnu funkciju i odziv sustava na ulaz i početne uvjete.


### Kako koristiti ovaj interaktivni primjer?
Podsjetimo se Kalmanove dekompozicije matrične strukture:

\begin{cases}
\underbrace{\begin{bmatrix} \dot{z}_{ro} \\ \dot{z}_{r\bar{o}} \\ \dot{z}_{no} \\ \dot{z}_{n\bar{o}} \end{bmatrix}}_{\dot{\textbf{z}}} = 
\begin{bmatrix} A_{ro} & 0 & A_{no,ro} & 0 \\ A_{ro,r\bar{o}} & A_{r\bar{o}} & A_{no,r\bar{o}} & A_{n\bar{o},r\bar{o}} \\ 0 & 0 & A_{no} & 0 \\ 0 & 0 & A_{no,n\bar{o}} & A_{n\bar{o}} \end{bmatrix} \underbrace{\begin{bmatrix} z_{ro} \\ z_{r\bar{o}} \\ z_{no} \\ z_{n\bar{o}} \end{bmatrix}}_{\textbf{z}} + 
\begin{bmatrix} B_{ro} \\ B_{r,\bar{o}} \\ 0 \\ 0 \end{bmatrix} \textbf{u} \\
\textbf{y} = \begin{bmatrix} C_{ro} & 0 & C_{no} & 0 \end{bmatrix} \textbf{z}
\end{cases}

modificirajte matrice $A$,$B$,$C$ i $D$ (prikazane ispod) i:
* pogledajte svojstvene vrijednosti kako biste vidjeli je li sustav stabilan ili nije;
* pogledajte prijenosnu funkciju da vidite je li se dogodilo poništavanje polova i nula;
* pogledajte kako sustav može imati divergentni slobodni odziv, ali i konvergentni izlaz;
* pokušajte shvatiti je li izgubljena potpuna osmostrivost ili upravljivost sustava;


i pokušajte stvoriti:
* sustav s osmotrivim / neosmotrivim nestabilnim modom;
* sustav s nedohvatljivim stanjem koje je nestabilno, ali ne daje divergentni slobodni odziv za bilo koje moguće početne uvjete.

In [2]:
#Preparatory Cell 

%matplotlib inline
import control
import control.matlab as cm
import numpy
from IPython.display import display, Markdown
import ipywidgets as widgets
import matplotlib.pyplot as plt

#print a matrix latex-like
def bmatrix(a):
     """Returns a LaTeX bmatrix - by Damir Arbula (ICCT project)

     :a: numpy array
     :returns: LaTeX bmatrix as a string
     """
     if len(a.shape) > 2:
         raise ValueError('bmatrix can at most display two dimensions')
     lines = str(a).replace('[', '').replace(']', '').splitlines()
     rv = [r'\begin{bmatrix}']
     rv += ['  ' + ' & '.join(l.split()) + r'\\' for l in lines]
     rv +=  [r'\end{bmatrix}']
     return ''.join(rv) #'\n'.join(rv)

#matrixWidget is a matrix looking widget built with a VBox of HBox(es) that returns a numPy array as value !
class matrixWidget(widgets.VBox):
    def updateM(self,change):
        for irow in range(0,self.n):
            for icol in range(0,self.m):
                self.M_[irow,icol] = self.children[irow].children[icol].value
                #print(self.M_[irow,icol])
        self.value = self.M_

    def dummychangecallback(self,change):
        pass
    
    
    def __init__(self,n,m):
        self.n = n
        self.m = m
        self.M_ = numpy.matrix(numpy.zeros((self.n,self.m)))
        self.value = self.M_
        widgets.VBox.__init__(self,
                             children = [
                                 widgets.HBox(children = 
                                              [widgets.FloatText(value=0.0, layout=widgets.Layout(width='90px')) for i in range(m)]
                                             ) 
                                 for j in range(n)
                             ])
        
        #fill in widgets and tell interact to call updateM each time a children changes value
        for irow in range(0,self.n):
            for icol in range(0,self.m):
                self.children[irow].children[icol].value = self.M_[irow,icol]
                self.children[irow].children[icol].observe(self.updateM, names='value')
        #value = Unicode('example@example.com', help="The email value.").tag(sync=True)
        self.observe(self.updateM, names='value', type= 'All')
        
    def setM(self, newM):
        #disable callbacks, change values, and reenable
        self.unobserve(self.updateM, names='value', type= 'All')
        for irow in range(0,self.n):
            for icol in range(0,self.m):
                self.children[irow].children[icol].unobserve(self.updateM, names='value')
        self.M_ = newM
        self.value = self.M_
        for irow in range(0,self.n):
            for icol in range(0,self.m):
                self.children[irow].children[icol].value = self.M_[irow,icol]
        for irow in range(0,self.n):
            for icol in range(0,self.m):
                self.children[irow].children[icol].observe(self.updateM, names='value')
        self.observe(self.updateM, names='value', type= 'All')        

                #self.children[irow].children[icol].observe(self.updateM, names='value')

             
#overlaod class for state space systems that DO NOT remove "useless" states (what "professor" of automatic control would do this?)
class sss(control.StateSpace):
    def __init__(self,*args):
        #call base class init constructor
        control.StateSpace.__init__(self,*args)
    #disable function below in base class
    def _remove_useless_states(self):
        pass

In [3]:
# Preparatory cell
import warnings
# In order to suppress the warning BadCoefficient
warnings.filterwarnings("ignore")

def MINREAL(A,B,C,D):
    ''' Trick in order to have a right cancellations without very small numbers '''
    sys = control.ss2tf(sss(A,B,C,D))
    sys = control.minreal(sys, verbose=False)
    z, p, k = cm.tf2zpk(sys.num[0][0],sys.den[0][0])
    num, den = cm.zpk2tf(z, p, k)
    sys = control.tf(num, den)
    return sys
    
A = numpy.matrix('-1 1 0 0; 0 -1 1 0; 0 0 -1 0; 0 1 -1 0')
B = numpy.matrix('0; 0; 1; -1')
C = numpy.matrix('1 0 0 1')
D = numpy.matrix('0')
X0 = numpy.matrix('0; 0; 0; 0')

Adef = [numpy.matrix('-1 1 4 0; 0 -0.5 3 0; 0 0 -2 0; 0 0 0 2'),
        numpy.matrix('-1 1 4 0; 0 -0.5 3 0; 0 0 -2 0; 0 0 -2 2'),
        numpy.matrix('-3 2 0 0; 0 -1 0 0; 0 1 1 0; 0 0 0 -1')] 
Bdef = [numpy.matrix('0; 0; 1; 0'),
        numpy.matrix('0; 0; 1; 0'),
        numpy.matrix('0; 1; 0; 0')] 
Cdef = [numpy.matrix('1 0 0 1'),
        numpy.matrix('1 0 0 0'),
        numpy.matrix('1 0 0 1')]

Aw = matrixWidget(4,4)
Aw.setM(A)
Bw = matrixWidget(4,1)
Bw.setM(B)
Cw = matrixWidget(1,4)
Cw.setM(C)
Dw = matrixWidget(1,1)
Dw.setM(D)
X0w = matrixWidget(4,1)
X0w.setM(X0)

In [4]:
# Misc
# default systems
sel_system = widgets.Dropdown(
                              options=['Ručno', 'Jedno nedohvatljivo stanje', 'Jedno neosmotrivo stanje', 'Jedno neosmotrivo stanje i jedno nedohvatljivo stanje'],
                              value='Jedno nedohvatljivo stanje',
                              description='',
                              disabled=False
)
#define type of ipout 
selu = widgets.Dropdown(
    options=['impuls', 'step', 'sinus', 'pravokutni val'],
    value='step',
    description='Tip ulaza:',
    disabled=False
)
# Define the values of the input
u = widgets.FloatSlider(
    value=1,
    min=0,
    max=20.0,
    step=0.1,
    description=r'$u$:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.1f',
)
period = widgets.FloatSlider(
    value=1,
    min=0.001,
    max=4,
    step=0.001,
    description='Period: ',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.2f',
)

#create dummy widget 
DW = widgets.FloatText(layout=widgets.Layout(width='0px', height='0px'))

#create button widget
START = widgets.Button(
    description='Test',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Test',
    icon='check'
)
                       
def on_start_button_clicked(b):
    #This is a workaround to have intreactive_output call the callback:
    #   force the value of the dummy widget to change
    if DW.value> 0 :
        DW.value = -1
    else: 
        DW.value = 1
    pass
START.on_click(on_start_button_clicked)

In [5]:
def main_callback(A, B, C, D, X0, u, period, selu, sel_system, DW):
    global Aw, Bw, Cw, Dw, X0w, Adef, Bdef, Cdef

    if sel_system == 'Jedno nedohvatljivo stanje':
        A = Adef[0]
        Aw.setM(A)
        B = Bdef[0]
        Bw.setM(B)
        C = Cdef[0]
        Cw.setM(C)
        D = 0
        Dw.setM(numpy.matrix('0'))
        X0w.setM(numpy.matrix([0, 0, 0, 0]).T)
        X0 = numpy.matrix([0, 0, 0, 0]).T
    elif sel_system == 'Jedno neosmotrivo stanje':
        A = Adef[1]
        Aw.setM(A)
        B = Bdef[1]
        Bw.setM(B)
        C = Cdef[1]
        Cw.setM(C)
        D = 0
        Dw.setM(numpy.matrix('0'))
        X0w.setM(numpy.matrix([0, 0, 0, 0]).T)
        X0 = numpy.matrix([0, 0, 0, 0]).T
    elif sel_system == 'Jedno neosmotrivo stanje i jedno nedohvatljivo stanje':
        A = Adef[2]
        Aw.setM(A)
        B = Bdef[2]
        Bw.setM(B)
        C = Cdef[2]
        Cw.setM(C)
        D = 0
        Dw.setM(numpy.matrix('0'))
        X0w.setM(numpy.matrix([0, 0, 0, 0]).T)
        X0 = numpy.matrix([0, 0, 0, 0]).T
        
        
    eig = numpy.linalg.eig(A)[0]
    R = control.ctrb(A,B)
    O = control.obsv(A,C)
    rankR = numpy.linalg.matrix_rank(R)
    rankO = numpy.linalg.matrix_rank(O)
    
    text = r'Svojstvene vrijednosti sustava su ' + str(eig[0]) + r', ' + str(eig[1])+ r', ' + str(eig[2]) + r' i ' + str(eig[3]) + '. '
    if rankO == 4:
        text = text + r'Sustav je osmotriv i '
    else:
        text = text + r'Sustav je neosmotriv i '
    if rankR == 4:
        text = text + r' dohvatljiv.'
    else:
        text = text + r' nije dohvatljiv. '
    
    
    sys = MINREAL(A,B,C,D)
    text = text + 'Prijenosna funkcija je: \n '
    display(Markdown(text))
    print(sys) #, tol=1e-3
    
    
    sys = sss(A,B,C,D)
    
    if (numpy.real([eig[0],eig[1],eig[2],eig[3]]) == [0, 0, 0, 0]).all():
        T = numpy.linspace(0,20,1000)
    else:
        if min(numpy.abs(numpy.real([eig[0],eig[1],eig[2],eig[3]]))) != 0:
            T = numpy.linspace(0,7*1/min(numpy.abs(numpy.real([eig[0],eig[1],eig[2],eig[3]]))),1000)
        else:
            T = numpy.linspace(0,7*1/max(numpy.abs(numpy.real([eig[0],eig[1],eig[2],eig[3]]))),1000)
    
    if selu == 'impuls': #selu
        U = [0 for t in range(0,len(T))]
        U[0] = u
        T, Y, X = control.forced_response(sys,T=T,U=U,X0=X0)
    if selu == 'step':
        U = [u for t in range(0,len(T))]
        T, Y, X = control.forced_response(sys,T=T,U=U,X0=X0)
    if selu == 'sinus':
        U = u*numpy.sin(2*numpy.pi/period*T)
        T, Y, X = control.forced_response(sys,T=T,U=U,X0=X0)
    if selu == 'pravokutni val':
        U = u*numpy.sign(numpy.sin(2*numpy.pi/period*T))
        T, Y, X = control.forced_response(sys,T=T,U=U,X0=X0)
    
    fig = plt.figure(figsize=(16,5))
    
    fig.add_subplot(121)
    plt.title('Početni odziv: stanja')
    plt.plot(T,X[0])
    plt.plot(T,X[1])
    plt.plot(T,X[2])
    plt.plot(T,X[3])
    plt.ylabel('Stanja')
    plt.xlabel('vrijeme [s]')
    plt.legend([r'$x_1$',r'$x_2$',r'$x_3$',r'$x_4$'])
    plt.grid()
    
    fig.add_subplot(122)
    plt.ylabel(r'$y$')
    plt.plot(T,Y)
    plt.xlabel('vrijeme [s]')
    plt.title('Početni odziv: izlaz')
    plt.grid()
    
    
alltogether = widgets.VBox([sel_system,
                            widgets.Label('   ',border=3),
                            widgets.HBox([widgets.Label('A:',border=3),  Aw, widgets.Label('   ',border=3),
                                          widgets.Label('B:',border=3),  Bw, widgets.Label('   ',border=3),
                                          widgets.Label('x0:',border=3),X0w,widgets.Label('   ',border=3)]),
                            widgets.Label('   ',border=3),
                            widgets.HBox([widgets.Label('C:',border=3),
                                          Cw,widgets.Label('   ',border=3),
                                          widgets.Label('D:',border=3),
                                          Dw,widgets.Label('   ',border=3),
                                          START]),
                            widgets.Label('   ',border=3),
                            widgets.HBox([selu, u, period])])
out = widgets.interactive_output(main_callback, {'A':Aw, 'B':Bw, 'C':Cw, 'D':Dw, 'X0':X0w, 'u':u, 'period':period, 'selu':selu, 'sel_system':sel_system, 'DW':DW})
out.layout.height = '520px'
display(out, alltogether)

Output(layout=Layout(height='520px'))